In [1]:
from tensorflow_model import train, load_image, predict
import Contrastive_Explanation_Method_master.Utils as utils
from Contrastive_Explanation_Method_master.aen_CEM import AEADEN
import random
import tensorflow as tf
import numpy as np
from dataset_handler import DatasetHandler
from tqdm import tqdm
import os

# 1. Load pretrained model

In [2]:
model = tf.keras.applications.VGG16(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000
)

# 2. Predict with pretrained model

In [3]:
# auffällig: Prediction ist deutlich unsicher als über PyTorch oder über Tensorflow 2.0 Model
# Wurden hier vielleicht einfach noch nicht die Wahrscheinlichkeiten normalisiert?
img = load_image("dog.jpg")
preds = predict(model, img)

(224, 224, 3)
(1, 224, 224, 3)
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Predicted: [('n02111889', 'Samoyed', 0.15781632), ('n02120079', 'Arctic_fox', 0.0915366), ('n02098286', 'West_Highland_white_terrier', 0.03576578)]


# 3. CEM

In [4]:
### Probleme:

# VGG laden -> tf 2  --> vielleicht musste nur zusätzlich zu tf auch keras in version 1.15 installiert werden, damit klappt
# CEM Implementierung -> tf 1

In [5]:
# Model class for the use in __init__ of AEDM() inspiried by class MNISTModel
class Model():
    
    def __init__(self, model, image_size, num_channels, num_labels):
        self.model = model
        self.image_size = image_size
        self.num_channels = num_channels
        self.num_labels = num_labels
        
    def predict(self, data):
        return self.model(data)

In [6]:
# Data class for the use of utils.generate_data
class Data():
    def __init__(self, test_data, test_labels):
        self.test_data = test_data
        self.test_labels = test_labels

In [7]:
# define data
test_data = np.expand_dims(img, axis=0)
target_label = preds.argmax()
test_labels = np.expand_dims(np.eye(1000)[target_label], axis=0)
image_id = 0


data = Data(test_data, test_labels)

In [8]:
# define paramters (was ist arg_b??)
arg_max_iter = 5
arg_b = 1 # binary_search_steps
arg_init_const = 0.1
arg_mode = "PP"
arg_kappa = 10
arg_beta = 0.1
arg_gamma = 0

In [9]:
with tf.Session() as sess:
    # load model after creating Session
    model_vgg = tf.keras.applications.VGG16(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000
    )
    
    model = Model(model=model_vgg, image_size=224, num_channels=3, num_labels=1000)

    random.seed(121)
    np.random.seed(1211)
    orig_prob, orig_class, orig_prob_str = utils.model_prediction(model, np.expand_dims(img, axis=0))
    target_label = orig_class
    print("Image:{}, infer label:{}".format(image_id, target_label))
    
    
    orig_img, target = utils.generate_data(data, image_id, target_label)
    


    attack = AEADEN(sess, model, mode = arg_mode, AE = lambda x: x, batch_size=1, kappa=arg_kappa, init_learning_rate=1e-2,
        binary_search_steps=arg_b, max_iterations=arg_max_iter, initial_const=arg_init_const, beta=arg_beta, gamma=arg_gamma)

    adv_img = attack.attack(orig_img, target)

    adv_prob, adv_class, adv_prob_str = utils.model_prediction(model, adv_img)
    delta_prob, delta_class, delta_prob_str = utils.model_prediction(model, orig_img-adv_img)

    INFO = "[INFO]id:{}, kappa:{}, Orig class:{}, Adv class:{}, Delta class: {}, Orig prob:{}, Adv prob:{}, Delta prob:{}".format(image_id, arg_kappa, orig_class, adv_class, delta_class, orig_prob_str, adv_prob_str, delta_prob_str)
    print(INFO)

    suffix = "id{}_kappa{}_Orig{}_Adv{}_Delta{}".format(image_id, arg_kappa, orig_class, adv_class, delta_class)
    arg_save_dir = "{}_ID{}_Gamma_{}".format(arg_mode, image_id, arg_gamma)
    os.system("mkdir -p Results/{}".format(arg_save_dir))
    utils.save_img(orig_img, "Results/{}/Orig_original{}.png".format(arg_save_dir, orig_class))
    utils.save_img(adv_img, "Results/{}/Adv_{}.png".format(arg_save_dir, suffix))
    utils.save_img(np.absolute(orig_img-adv_img)-0.5, "Results/{}/Delta_{}.png".format(arg_save_dir, suffix))

    sys.stdout.flush()

AttributeError: module 'tensorflow' has no attribute 'Session'